In [2]:
import numpy as np
from scipy.special import jv
import copy
from hamiltonian import hamiltonian_c, setup_scaled_H
from numba import jit, autojit
import math

In [3]:
b_field = 0.0           #BField
n_tot = 40000            #TotalAtomNumber
m = -4                #Magnetization
mag_range = 7         #MagRange
atom_range = 40       #AtomRange
spinor_phase = 0.0      #SpinorPhase
n_0 = 39996              #N_0 numbers tarting in m=0
c = 24             #C_init in Hz
q = -2.5
dt = .04/30
alpha = np.sqrt(n_0)*np.exp(np.complex(0,0))

In [76]:
def calc_n0_vals(psi,num_atoms,k):
    n0 = np.sum((num_atoms-2*k) * abs(psi)**2 )  
    n0sqr = np.sum((num_atoms-2*k)**2 * abs(psi)**2)  
    n0var = n0sqr - np.power(n0,2)
    return n0, n0sqr,n0var


@autojit
def calc_n0_vals_fast(psi,num_atoms):

    n0 = 0
    n0sqr = 0
    for k in range(len(psi)):
        n0 += (num_atoms-2*k) * abs(psi[k])**2 
        n0sqr += (num_atoms-2*k)**2 * abs(psi[k])**2
    n0var = n0sqr - n0**2
    return n0, n0sqr,n0var

In [77]:
num_atoms = 40000
psi = np.zeros(int(num_atoms/2)+1,dtype = complex)
for i in range(len(psi)):
    x = np.random.rand()
    y = np.random.rand()
    psi[i] = np.complex(x,y)
k = np.asarray([i for i in range(len(psi))])

In [78]:
%timeit calc_n0_vals(psi,num_atoms,k)

1000 loops, best of 3: 836 µs per loop


In [79]:
%timeit calc_n0_vals_fast(psi,num_atoms)

The slowest run took 4883.09 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 209 µs per loop


In [80]:
calc_n0_vals(psi,num_atoms,k)==calc_n0_vals_fast(psi,num_atoms)

False

In [81]:
calc_n0_vals_fast(psi,num_atoms)

(268615186.276457, 7178663334809.932, -7.214693963500088e+16)

In [82]:
calc_n0_vals(psi,num_atoms,k)

(268615186.27645773, 7178663334809.9395, -72146939635001264.0)